In [1]:
import androguard
from androguard import misc

from datetime import datetime


## Getting started with a, d, dx (analysis objects)

In [2]:
# a --> APK object
# d --> array of DalvikVMFormat object
# dx --> Analysis object

now = datetime.now()
a, d, dx = misc.AnalyzeAPK("data/Pinterest.apk")
print('time took for decompiling: ', (datetime.now() - now))

Requested API level 29 is larger than maximum we have, returning API level 28 instead.


time took for decompiling:  0:01:29.248796


## Crossreferences (XREFs) and what they are
- XREFs are the main thing that Analysis object provides (dx)
    - XREFs are generated for Classes, Methods, Fields and Strings

### XREF of methods in a class

In [3]:
dx.classes

{'La;': <analysis.ClassAnalysis La;>,
 'Landroid/support/v4/media/MediaBrowserCompat$MediaItem$a;': <analysis.ClassAnalysis Landroid/support/v4/media/MediaBrowserCompat$MediaItem$a;>,
 'Landroid/support/v4/media/MediaBrowserCompat$MediaItem;': <analysis.ClassAnalysis Landroid/support/v4/media/MediaBrowserCompat$MediaItem;>,
 'Landroid/support/v4/media/MediaDescriptionCompat$a;': <analysis.ClassAnalysis Landroid/support/v4/media/MediaDescriptionCompat$a;>,
 'Landroid/support/v4/media/MediaDescriptionCompat;': <analysis.ClassAnalysis Landroid/support/v4/media/MediaDescriptionCompat;>,
 'Landroid/support/v4/media/MediaMetadataCompat$a;': <analysis.ClassAnalysis Landroid/support/v4/media/MediaMetadataCompat$a;>,
 'Landroid/support/v4/media/MediaMetadataCompat;': <analysis.ClassAnalysis Landroid/support/v4/media/MediaMetadataCompat;>,
 'Landroid/support/v4/media/RatingCompat$a;': <analysis.ClassAnalysis Landroid/support/v4/media/RatingCompat$a;>,
 'Landroid/support/v4/media/RatingCompat;': 

In [18]:
# query all called classes from one class, returns ClassAnalysis object
class_to_query = 'Landroidx/appcompat/widget/AppCompatSpinner$SavedState;'
display(dx.classes[class_to_query])
type(dx.classes[class_to_query])


<analysis.ClassAnalysis Landroidx/appcompat/widget/AppCompatSpinner$SavedState;>

androguard.core.analysis.analysis.ClassAnalysis

In [19]:
# XREFs for methods
# get_method gets the methods inside the class 
## get_xref_to --> Returns a list of three tuples cotaining the class, method and offset of
##                 the call, which are called by this method.

for meth in dx.classes[class_to_query].get_methods():
    print("inside method {}".format(meth.name))
    for _, call, _ in meth.get_xref_to():
        print("  calling -> {} -- {}".format(call.class_name, call.name))

inside method <init>
  calling -> Landroid/os/Parcel; -- readByte
  calling -> Landroid/view/View$BaseSavedState; -- <init>
inside method <clinit>
  calling -> Landroidx/appcompat/widget/AppCompatSpinner$SavedState$a; -- <init>
inside method <init>
  calling -> Landroid/view/View$BaseSavedState; -- <init>
inside method writeToParcel
  calling -> Landroid/os/Parcel; -- writeByte
  calling -> Landroid/view/View$BaseSavedState; -- writeToParcel


In [20]:
# XREFs for methods
# Doing this the other way around 
## get_xref_from --> Returns a list of three tuples cotaining the class, method and offset of
##                  the call, from where this object was called.
for meth in dx.classes[class_to_query].get_methods():
    print("inside method {}".format(meth.name))
    for _, call, _ in meth.get_xref_from():
        print("  called by -> {} -- {}".format(call.class_name, call.name))


inside method <init>
  called by -> Landroidx/appcompat/widget/AppCompatSpinner$SavedState$a; -- createFromParcel
inside method <clinit>
inside method <init>
  called by -> Landroidx/appcompat/widget/AppCompatSpinner; -- onSaveInstanceState
inside method writeToParcel


### XREF of usage of strings

In [7]:
dx.strings

{'MediaItem{': <analysis.StringAnalysis 'MediaItem{'>,
 'mFlags=': <analysis.StringAnalysis 'mFlags='>,
 ', mDescription=': <analysis.StringAnalysis ', mDescription='>,
 'android.support.v4.media.description.MEDIA_URI': <analysis.StringAnalysis 'android.support.v4.m'...>,
 'android.support.v4.media.description.NULL_BUNDLE_FLAG': <analysis.StringAnalysis 'android.support.v4.m'...>,
 ', ': <analysis.StringAnalysis ', '>,
 'android.media.metadata.TITLE': <analysis.StringAnalysis 'android.media.metada'...>,
 'android.media.metadata.ARTIST': <analysis.StringAnalysis 'android.media.metada'...>,
 'android.media.metadata.DURATION': <analysis.StringAnalysis 'android.media.metada'...>,
 'android.media.metadata.ALBUM': <analysis.StringAnalysis 'android.media.metada'...>,
 'android.media.metadata.AUTHOR': <analysis.StringAnalysis 'android.media.metada'...>,
 'android.media.metadata.WRITER': <analysis.StringAnalysis 'android.media.metada'...>,
 'android.media.metadata.COMPOSER': <analysis.StringAna

In [8]:
# XREFs for strings
## maybe we will find some string interesting and we wanna know where it is used
## returns a StringAnalysis object
string_to_query = 'android.media.metadata.DISC_NUMBER'
display(dx.strings[string_to_query])
type(dx.strings[string_to_query])

<analysis.StringAnalysis 'android.media.metada'...>

androguard.core.analysis.analysis.StringAnalysis

In [9]:
# call xref_from() to get the usage of the string
for _, meth in dx.strings[string_to_query].get_xref_from():
    print("Used in: {} -- {}".format(meth.class_name, meth.name))
    
    
# so we know that the string_to_query is used in the a method in Laaan; class

Used in: Landroid/support/v4/media/MediaMetadataCompat; -- <clinit>


In [10]:
# calling xref_to()
# call xref_from() to get the usage of the string
for _, meth in dx.strings[string_to_query].get_xref_to():
    print("Used in: {} -- {}".format(meth.class_name, meth.name))

AttributeError: 'StringAnalysis' object has no attribute 'get_xref_to'

### XREF for fields
- fields in Java are variables

In [ ]:
# dx.find_fields() --> find fields by regex

for field in dx.find_fields(classname=class_to_query, fieldname=""):
    print("Field: {}".format(field.name))
    for _, meth in field.get_xref_read():
        print("  read in {} -- {}".format(meth.class_name, meth.name))
    for _, meth in field.get_xref_write():
        print("  write in {} -- {}".format(meth.class_name, meth.name))

## Getting Control Flow Graphs

<analysis.ClassAnalysis Landroidx/appcompat/widget/ActionBarContextView$a;>